# Section Intro

In this section we will look at
* Memory References
* What variables really are (in the context of memory references)
* Memory Management
* Reference Counting, Garbage Collection (note that these 2 are not the same thing)
* Dynamic vs Static Typing
* Mutability and Immutability
* Shared References
* Variable equality
* In Python everything is an object

# 1. Variables and Memory References

* Memory can be thought of as a series of slots/boxes and we can store and retrieve data from those boxes.
* We need unique address for each of those slots/boxes (typically just numbers)
* When we store data im memory addresses we may actually use more than 1 slot at a time
* Storing and Retrieving object from the heap is taken care of for us by "Python Memory Manager"

![image](img/python_memory.png)

* When you execute "my_var_1 = 10" Python creates an object in memory at some address, say 1000, and it stores the value 10 inside that object
* my_var_1 is simply a name/alias for the memory address where that object is stored or the starting address of the object if it overflows to multiple slots
* Hence **my_var_1 references the object at 0x1000**
* Note that my_var_1 != 10; my_var_1 = 0x1000 (in this case); But 0x1000 represents the memory addr of the data we're actually interested in
* Similarly, **my_var_2 references the object at 0x1002**

## Variables in Python are always references to objects in memory

## id()
* In Python, we can find out the memory address referenced by a variable by using **id()** function
* id() function returns a base 10 number which we can covert to hexadecimal number using **hex()** function

In [1]:
my_var = 10

In [2]:
print(my_var)

10


What actually happened is that Python looked at my_var. It then looked at what is the memory address that my_var is referencing. It found that memory address. It went to the memory, retrieved the data from memory and brought it back to be displayed.

In [3]:
print(id(my_var))

138428361425424


In [4]:
print(hex(id(my_var)))

0x7de65d4f4210


In [5]:
greeting = 'hello'

In [6]:
print(greeting)

hello


In [7]:
print(id(greeting))

138428276935344


In [8]:
print(hex(id(greeting)))

0x7de658460ab0


# 2. Reference Counting

We can start keeping track of these objects that are created in memory by keeping track of their memory address and how many variables are pointing to that same object.

![Reference_counting_1](img/Reference_counting_1.png)

Remember, when we say "oter_var = my_var" we are actually taking the reference of my_var and assigning that reference (0x1000) to other_var. The reference counter now goes to 2.

Now suppose "my_var" goes away: either it falls out of scope or we assign it to a different object in memory, then the reference goes away and reference count goes down to 1.

![Reference_counting_2](img/Reference_counting_2.png) 

Let say "other_var" also goes away. Now the reference count drops down to 0.

![Reference_counting_3](img/Reference_counting_3.png)

* At this point Python Memory manager recognizes that and says there's no references to this object so I don't need it anymore and it throws away the object.
* The space that the object previously used can now be reused by Python when it's running our program.

**This is called reference counting** and this is something that Python Memory Manager does for us automatically.

## How to find reference count of a variable

**Option 1** - we can use **sys.getrefcount(my_var)** function provided by **sys module** for this purpose

Note that passing "my_var" to getrefcount() creates an extra reference!
This happens because the memory address in my_var gets assigned to the parameter used in the getrefcount function (***variables are passed by reference in Python***). The scope of this parameter is till the end of this function

**Option 2** - ***<u>ctypes.c_long.from_address(address).value</u>***

This is at lower level. The difference between the 2 functions is that in ths fuction we directly pass the memory address(an integer), not a reference. Therefore it doesn't affect reference count.

In [9]:
import sys

In [10]:
a = [1, 2, 3]

We'll see in upcoming lessons why am I not using an integer here - because you might be surprised at the answer we get.

In [11]:
id(a)

131373005329600

In [12]:
sys.getrefcount(a)

2

In [13]:
import ctypes

In [14]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [16]:
ref_count(id(a))

1

* Here id of a is getting evaluated first. Indeed whe id function is running, then the reference count to that memory address is 2.
* But id then finishes running and returns the memory address. So by the time we call ref_count, id has finished running and it has released it's pointer to that memory address.
* This is exactly the same as below

In [17]:
ref_count(131373005329600)

1

In [18]:
b = a

In [20]:
id(b)

131373005329600

**Note** that the memory address of variables a and b is the same.

In [21]:
ref_count(id(a))

2

In [22]:
c = a

In [23]:
ref_count(id(a))

3

Now I change c to something else

In [25]:
c = 10

In [26]:
ref_count(id(a))

2

Now I assign None to b

In [27]:
b = None

In [28]:
ref_count(id(a))

1

**Now,** 
* I am going to store the memory address of a in variable "a_id"
* Set a = None
* then check refcount of a_id, i.e., check the reference count of the object at particular memory address

In [29]:
a_id = id(a)
a = None
ref_count(a_id)

1

In [30]:
ref_count(a_id)

1

In [31]:
ref_count(a_id)

0

* I know that the answers we got here is not what we had expected. 1st it showed 1, 2nd time it showed 1 again, then 3rd time it showed 0 ang subsequent eexecution it might show some other random number as well.
* We will get into lot more details of why this happens later.

When the last reference to that memory address was dropped when we set a = None and memory manager frees up that memory address and essentially tosses the object away and that memory address becomes available for something else.

Typically in Python we don't work with memory addresses unless in very rare case where you are trying to debug something. This is just to understand what happens behind the scene.

### 3. Garbage Collection